# Carry Trade

## Imports

In [7]:
# <include-carry_trade/utils.py>

In [69]:
# <imports>
import numpy as np
import pandas as pd
import plotly.io as pio

from carry_trade import utils

pd.options.plotting.backend = "plotly"
pio.templates.default = "seaborn"

## Summary

* Definitely look at maximum draw down

In [2]:
start_date = "2014-12-01"
tickers = ["THA", "ROU", "JPN", "IDN"]
libors = ["JPY3MTD156N"]
currencies = ["THB", "RON", "JPY", "IDR"]

## Thai Baht YC/THA

In [9]:
dfs_yc = utils.load_yc(tickers)
dfs_fx = utils.load_fx(currencies)
dfs_libor = utils.load_libor(libors)

In [134]:
dfs_yc = {t: pd.read_csv(f"data/df_{t}.csv") for t in tickers}
dfs_fx = {c: pd.read_csv(f"data/df_fx_{c}.csv") for c in currencies}
dfs_libor = {l: pd.read_csv(f"data/df_libor_{l}.csv") for l in libors}
for d in [dfs_yc, dfs_fx, dfs_libor]:
    for df in d.values():
        df["date"] = pd.to_datetime(df.date)
        df.set_index("date", inplace=True)



for k in dfs_yc:
    dfs_yc[k] = dfs_yc[k].reindex(pd.date_range(dfs_yc[k].index.min(), dfs_yc[k].index.max())).iloc[:, :-1]
    dfs_yc[k] = dfs_yc[k].interpolate(axis=0).interpolate(axis=1)

for k in dfs_fx:
    dfs_fx[k] = dfs_fx[k].reindex(pd.date_range(dfs_fx[k].index.min(), dfs_fx[k].index.max())).iloc[:, :-1]
    dfs_fx[k] = dfs_fx[k].interpolate()



* change value in lending market
* change in fx rates between lending market and borrowing market
* change in fx rates between lending market and home market

In [32]:
strategies = [
    {"yc_L": "THA", "fx_B": "JPY", "fx_L": "THB", "libor": "JPY3MTD156N", "leverage": 0.8},
    {"yc_L": "ROU", "fx_B": "JPY", "fx_L": "RON", "libor": "JPY3MTD156N", "leverage": 0.8},
    {"yc_L": "IDN", "fx_B": "JPY", "fx_L": "IDR", "libor": "JPY3MTD156N", "leverage": 0.8},
]

In [219]:
strategy = strategies[2]
date_range = pd.date_range("2015-01-01", "2021-04-26", freq="7D")

In [220]:
df_ret, df_profit = utils.run_strategy(
    **strategy,
    date_range=date_range,
    dfs_yc=dfs_yc,
    dfs_fx=dfs_fx,
    dfs_libor=dfs_libor   
)

In [221]:
(df_profit[["lend", "fx_LB", "fx_BH", "interest"]] / K_H0).plot(kind="bar", barmode="relative")

In [222]:
fig = utils.make_components_chart(
    **strategy,
    date_range=date_range,
    dfs_yc=dfs_yc,
    dfs_fx=dfs_fx,
    dfs_libor=dfs_libor
)
fig.show()

In [223]:
utils.make_returns_chart(df_ret)

In [216]:
df_profit

,lend,fx_LB,fx_BH,interest,total
date,,,,,
2015-01-08,132663.904142,4985.640807,10671.271993,-952.217778,114280.429027
2015-01-15,23373.020598,6848.742240,57142.071043,-948.888889,-171847.514104
2015-01-22,-63360.463958,8876.227640,-35874.083537,-937.782222,123314.846257
2015-01-29,15173.653978,-6286.615255,3554.366068,-928.884444,-31455.117273
2015-02-05,-11269.380510,6249.235477,11907.666044,-938.886667,-28633.556446
...,...,...,...,...,...
2021-03-25,37217.153456,-24251.963445,-1248.427990,-653.597778,-80153.436124
2021-04-01,36639.599072,-13988.489918,-31862.003675,-660.333333,93239.084289
2021-04-08,9253.002109,-7148.280935,18794.036655,-666.291111,-102370.172976


In [201]:
ycs = pd.concat([zcb_L0, zcb_L1], axis=1)
ycs.columns = ["zcb_0", "zcb_1"]
ycs.plot()

### Beginning of the Week

1. Borrow $8MM USD notional JPY
2. Exchange $2MM USD for JPY
3. Exchange $10MM USD notional JPY for Thai Baht
4. Buy $10MM USD notional Thai Government Bonds

### End of the Week

1. Sell Thai Government Bonds
2. Exchange Thai Bhat for JPY
3. Repay JPY borrowings
4. Convert profit in JPY to USD for accounting

### Assumptions / Questions

1. Yield curve maturities of one year or less have no coupons
2. Is buying a notional amount based on zero curve rate the same thing as buying based on the swap rate and repricing all of the coupons based on the new zero curve rate?

In [86]:
if False:
    ticker = tickers[1]
    data = utils.fetch_ticker(ticker, database_code="YC").to_csv(f"data/df_{ticker}.csv", index=False)

    currency = currencies[3]
    (utils.fetch_ticker(currency, database_code="CUR", query_params={"start_date": start_date})
    .to_csv(f"data/df_fx_{currency}.csv", index=False))

# libor = libors[0]
# (utils.fetch_ticker(libor, database_code="FRED", query_params={"start_date": start_date})
# .to_csv(f"data/df_libor_{libor}.csv", index=False))

In [322]:
week_start = pd.Timestamp("2021-04-21")
week_end = week_start + pd.DateOffset(days=7)
notional_start_USD = float(10e6)
capital_start_USD = float(2e6)
borrow_start_USD = notional_start_USD - capital_start_USD
borrow_start_USD

8000000.0

Borrow $8MM value JPY

In [321]:
borrow_rate = dfs_libor["JPY3MTD156N"].loc[week_start].value
fx_rate_start_borrow_USD = dfs_fx["JPY"].loc[week_start].rate
borrow_start_borrow = borrow_start_USD * fx_rate_start_borrow_USD

Exchange $2MM USD for JPY

In [256]:
capital_start_borrow = capital_start_USD * fx_rate_start_borrow_USD

Exchange $10MM USD JPY for Thai Baht

In [287]:
fx_rate_start_lend_USD = dfs_fx["THB"].loc[week_start].rate
fx_rate_start_lend_borrow = fx_rate_start_lend_USD / fx_rate_start_borrow_USD
value_start_lend = (borrow_start_borrow + capital_start_borrow) * fx_rate_start_lend_borrow
value_start_lend

313199980.0

Buy $10MM USD value Thai Government Bonds

In [335]:
dfs_yc["THA"].reindex

,1-month,3-month,6-month,1-year,2-year,3-year,4-year,5-year,6-year,7-year,8-year,9-year,10-year,15-year,ticker
date,,,,,,,,,,,,,,,
2014-12-01,2.0130,2.0200,2.0200,2.010,2.075,2.095,2.232,2.310,2.383,3.301,2.696,2.738,3.304,3.231,THA
2014-12-02,2.0110,2.0200,2.0200,2.000,2.079,2.078,2.235,2.304,2.374,3.301,2.693,2.738,3.304,3.198,THA
2014-12-03,2.0090,2.0100,2.0100,2.000,2.076,2.085,2.219,2.328,2.386,3.301,2.687,2.753,2.919,3.237,THA
2014-12-04,2.0090,2.0100,2.0100,2.000,2.074,2.091,2.243,2.329,2.403,3.301,2.702,2.752,2.911,3.227,THA
2014-12-05,2.0090,2.0100,2.0100,2.000,2.074,2.091,2.243,2.329,2.403,3.301,2.702,2.752,2.911,3.227,THA
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2021-04-23,0.3012,0.3252,0.4050,0.446,0.413,0.686,0.905,0.993,NaN,NaN,1.548,NaN,1.850,2.127,THA
2021-04-26,0.3003,0.3253,0.4089,0.446,0.468,0.676,0.888,0.970,NaN,NaN,1.517,NaN,1.792,2.091,THA
2021-04-27,0.3002,0.3268,0.4113,0.449,0.476,0.694,0.905,0.996,NaN,NaN,1.553,NaN,1.828,2.121,THA


In [282]:
dfs_yc["THA"].loc[week_start].index

Index(['1-month', '3-month', '6-month', '1-year', '2-year', '3-year', '4-year',
       '5-year', '6-year', '7-year', '8-year', '9-year', '10-year', '15-year',
       'ticker'],
      dtype='object')

In [294]:
scr_start = utils.get_spot_curve_tenors(dfs_yc["THA"].loc[week_start][2:8])
zcb_start = utils.compute_zcb_curve(scr_start.to_frame())

In [295]:
zcb

,rate
0.5,0.004091
1.0,0.004518
2.0,0.004347
3.0,0.006842
4.0,0.009038
5.0,0.009887


In [313]:
utils.bond_price(zcb.rate, 0.984/100, 5.0)

0.9999992354379328

In [326]:
N = value_start_lend / np.exp(-zcb.iloc[-1] * zcb.index[-1])
N

rate    3.290725e+08
Name: 5.0, dtype: float64

In [315]:
scr_end = utils.get_spot_curve_tenors(dfs_yc["THA"].loc[week_end][2:8])
zcb_end = utils.compute_zcb_curve(scr_end.to_frame())
zcb_end

,rate
0.5,0.004143
1.0,0.004498
2.0,0.004758
3.0,0.007092
4.0,0.009187
5.0,0.010057


In [320]:
zcb_end_interp = utils.get_zcb_interp(zcb_end)
zcb_end_interp

0.480769    0.004143
0.980769    0.004484
1.980769    0.004753
2.980769    0.007047
3.980769    0.009147
4.980769    0.010040
dtype: float64

In [328]:
V = N * utils.bond_price(zcb_end_interp, 0.984/100, zcb_end_interp.index.values[-1])
V

rate    3.288879e+08
Name: 5.0, dtype: float64

In [ ]:
for row in 

In [ ]:
V = 

In [258]:
yield_start_lend = dfs_yc["THA"].loc[week_start, "5-year"] / 100
yield_start_lend

0.00984

In [236]:
s_cash_flow = utils.get_bond_cash_flows(yield_start_lend, notional_start_lend, week_start)
s_cash_flow

coupon_date
2021-07-21    0.00246
2021-10-21    0.00246
2022-01-21    0.00246
2022-04-21    0.00246
2022-07-21    0.00246
2022-10-21    0.00246
2023-01-21    0.00246
2023-04-21    0.00246
2023-07-21    0.00246
2023-10-21    0.00246
2024-01-21    0.00246
2024-04-21    0.00246
2024-07-21    0.00246
2024-10-21    0.00246
2025-01-21    0.00246
2025-04-21    0.00246
2025-07-21    0.00246
2025-10-21    0.00246
2026-01-21    0.00246
2026-04-21    1.00246
Name: cash_flow, dtype: float64

Sell Thai Government Bonds

In [242]:
df_spot_curve = utils.interpolate_spot_rates_curve(dfs_yc["THA"].loc[week_end].iloc[:-1], start_date=week_start)
df_spot_curve


,rate
coupon_date,
2021-07-21,0.003287
2021-10-21,0.004145
2022-01-21,0.004322
2022-04-21,0.004500
2022-07-21,0.004565
2022-10-21,0.004630
2023-01-21,0.004695
2023-04-21,0.004760
2023-07-21,0.005340


In [243]:
dfs_yc["THA"]

,1-month,3-month,6-month,1-year,2-year,3-year,4-year,5-year,6-year,7-year,8-year,9-year,10-year,15-year,ticker
date,,,,,,,,,,,,,,,
2014-12-01,2.0130,2.0200,2.0200,2.010,2.075,2.095,2.232,2.310,2.383,3.301,2.696,2.738,3.304,3.231,THA
2014-12-02,2.0110,2.0200,2.0200,2.000,2.079,2.078,2.235,2.304,2.374,3.301,2.693,2.738,3.304,3.198,THA
2014-12-03,2.0090,2.0100,2.0100,2.000,2.076,2.085,2.219,2.328,2.386,3.301,2.687,2.753,2.919,3.237,THA
2014-12-04,2.0090,2.0100,2.0100,2.000,2.074,2.091,2.243,2.329,2.403,3.301,2.702,2.752,2.911,3.227,THA
2014-12-05,2.0090,2.0100,2.0100,2.000,2.074,2.091,2.243,2.329,2.403,3.301,2.702,2.752,2.911,3.227,THA
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2021-04-23,0.3012,0.3252,0.4050,0.446,0.413,0.686,0.905,0.993,NaN,NaN,1.548,NaN,1.850,2.127,THA
2021-04-26,0.3003,0.3253,0.4089,0.446,0.468,0.676,0.888,0.970,NaN,NaN,1.517,NaN,1.792,2.091,THA
2021-04-27,0.3002,0.3268,0.4113,0.449,0.476,0.694,0.905,0.996,NaN,NaN,1.553,NaN,1.828,2.121,THA


In [248]:
zero_rates = utils.compute_zcb_curve_cse(df_spot_curve, week_end)

2021-07-21 00:00:00 2021-04-28 00:00:00 0.003287 0.003287
2021-10-21 00:00:00 2021-07-21 00:00:00 0.004144999999999999 0.004144999999999999
2022-01-21 00:00:00 2021-10-21 00:00:00 0.0043225 0.0043225
2022-04-21 00:00:00 2022-01-21 00:00:00 0.0045000000000000005 0.0045000000000000005
2022-07-21 00:00:00 2022-04-21 00:00:00 0.0045650000000000005 0.0045650000000000005
2022-10-21 00:00:00 2022-07-21 00:00:00 0.00463 0.004684010780397242
2023-01-21 00:00:00 2022-10-21 00:00:00 0.0046949999999999995 0.004736047533320364
2023-04-21 00:00:00 2023-01-21 00:00:00 0.0047599999999999995 0.004804547544265953
2023-07-21 00:00:00 2023-04-21 00:00:00 0.00534 0.0053884162792833135
2023-10-21 00:00:00 2023-07-21 00:00:00 0.00592 0.005966402810671537
2024-01-21 00:00:00 2023-10-21 00:00:00 0.006499999999999999 0.006545225259500992
2024-04-21 00:00:00 2024-01-21 00:00:00 0.0070799999999999995 0.007131498883266847
2024-07-21 00:00:00 2024-04-21 00:00:00 0.007597499999999999 0.007655090581733816
2024-10-21 

In [245]:
zero_rates

,rate,tenor,spot_rate
coupon_date,,,
2021-07-21,0.003287,0.230137,0.003287
2021-10-21,0.004145,0.482192,0.004145
2022-01-21,0.004322,0.734247,0.004322
2022-04-21,0.004500,0.980822,0.004500
2022-07-21,0.004565,1.230137,0.004565
2022-10-21,0.004684,1.482192,0.004630
2023-01-21,0.004736,1.734247,0.004695
2023-04-21,0.004805,1.980822,0.004760
2023-07-21,0.005388,2.230137,0.005340


In [246]:
(np.exp(-zero_rates.rate.values * zero_rates.tenor.values) * s_cash_flow.values).sum() * 100

99.9167116984626

In [160]:
prior_date = "2022-04-28"
coupon_rate = zero_rates.loc["2022-07-28"].rate * 0.25
tenor = zero_rates.loc["2022-07-28"].tenor
zero_rate = (
    -np.log(
        (
            1
            - coupon_rate
            * np.exp(
                -zero_rates.loc[:prior_date].rate
                * zero_rates.loc[:prior_date].tenor
            ).sum()
        )
        / (1 + coupon_rate)
    )
    / tenor
)
zero_rate, coupon_rate

(0.004566174470713188, 0.0011413966736710417)

In [154]:
times = np.arange(tenor - 0.5, 0, step=-0.5)[::-1]
coupon_half_yr = 0.5 * coupon_rate / 0.25
times, coupon_half_yr

(array([0.25616438, 0.75616438, 1.25616438, 1.75616438, 2.25616438,
        2.75616438, 3.25616438]),
 0.004323820007303257)

In [155]:
times

array([0.25616438, 0.75616438, 1.25616438, 1.75616438, 2.25616438,
       2.75616438, 3.25616438])

In [156]:
z = np.interp(
    times, zero_rates.tenor, zero_rates.rate
)
z

array([0.00331032, 0.00432447, 0.00456717, 0.0046848 , 0.00535795,
       0.00650584, 0.00761816])

In [31]:
t_rates = yc_end.iloc[:5]
t_times = (t_rates.index - week_end).days / 364
t_rates, t_times

(             spot_rate
 coupon_date           
 2021-07-28     0.32870
 2021-10-28     0.41450
 2022-01-28     0.43225
 2022-04-28     0.45000
 2022-07-28     0.45650,
 Float64Index([              0.25, 0.5027472527472527, 0.7554945054945055,
               1.0027472527472527, 1.2527472527472527],
              dtype='float64', name='coupon_date'))

In [267]:
t_rates.spot_rate[-1] * np.exp(t_rates.spot_rate[:-1] * t_times[:-1]).sum()

0.4883087278007508

In [260]:
cash_flows

array([3.0818878e+08, 3.0818878e+08, 3.0818878e+08, 3.0818878e+08,
       3.0818878e+08, 3.0818878e+08, 3.0818878e+08, 3.0818878e+08,
       3.0818878e+08, 3.0818878e+08, 3.0818878e+08, 3.0818878e+08,
       3.0818878e+08, 3.0818878e+08, 3.0818878e+08, 3.0818878e+08,
       3.0818878e+08, 3.0818878e+08, 3.0818878e+08, 6.2138876e+08])

In [19]:
spot_curves = pd.DataFrame({"JPN": dfs_yc[tickers[2]].set_index("date").loc[start_date] .iloc[:10],
"IDN": dfs_yc[tickers[3]].set_index("date").loc[start_date] .iloc[:10]
})
spot_curves.index = spot_curves.index.map(lambda x: x.split("-")[0]).astype(int)
spot_curves.index.name="Tenor"
spot_curves = spot_curves.astype(float)
spot_curves.plot()